In [1]:
import os
import cltk
import re
import epitomes
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
epitome_path = '../epitomes'
epitome_texts, epitome_numbers = epitomes.load_epitomes(epitome_path)

In [3]:
cltk_lat = cltk.NLP('lat', suppress_banner=True)
cltk_lat.pipeline.processes.remove(cltk.lexicon.processes.LatinLexiconProcess)
cltk_lat.pipeline.processes.remove(cltk.embeddings.processes.LatinEmbeddingsProcess)

In [4]:
epitome_pos, epitome_emb, epitome_tokens = epitomes.generate_text_vectors(epitome_texts, cltk_lat)

  0%|          | 0/1416 [00:00<?, ?it/s]

Unrecognized value 'Perf' for UD feature 'Tense'.
If you believe this is not an error in the dependency parser, please raise an issue at <https://github.com/cltk/cltk/issues> and include a short text to reproduce the error.

Unrecognized value 'Perf' for UD feature 'Tense'.
If you believe this is not an error in the dependency parser, please raise an issue at <https://github.com/cltk/cltk/issues> and include a short text to reproduce the error.

Unrecognized value 'Perf' for UD feature 'Tense'.
If you believe this is not an error in the dependency parser, please raise an issue at <https://github.com/cltk/cltk/issues> and include a short text to reproduce the error.

Unrecognized value 'Perf' for UD feature 'Tense'.
If you believe this is not an error in the dependency parser, please raise an issue at <https://github.com/cltk/cltk/issues> and include a short text to reproduce the error.

Unrecognized value 'Perf' for UD feature 'Tense'.
If you believe this is not an error in the depende

In [6]:
stop_list = ['', '\n', 'doceo', 'liber', 'omnis', 'alius']
most_common_tokens = epitomes.token_counter(epitome_tokens, stop_list).most_common(20)
most_common_tokens = [token for token, count in most_common_tokens]

In [7]:
epitome_pos_tokens = epitomes.combine_pos_most_common_tokens(epitome_tokens, epitome_pos, most_common_tokens)

In [8]:
ngrams = [2, 3, 4]
ranks = [100, 200, 300]
tfidf_dict = {}
feat_dict = {}
for n in ngrams:
    tfidf_dict[n] = {}
    feat_dict[n] = {}
    for r in ranks:
        word_vec, word_feature_names = epitomes.word_tfidf(epitome_pos_tokens, n, r)
        char_vec, char_feature_names = epitomes.char_tfidf(epitome_tokens, n, r)
        feat_dict[n][r] = np.append(word_feature_names, char_feature_names)
        tfidf_dict[n][r] = np.append(word_vec.toarray(), char_vec.toarray(), axis=1)

In [18]:
for ngram in feat_dict:
    for rank in feat_dict[ngram]:
        with open(f'csvfiles/featnames_n{ngram}_r{rank}.txt', 'w', encoding='utf-8') as f:
            for feat in feat_dict[ngram][rank]:
                f.write(f'{feat}\n')

In [31]:
for ngram in tfidf_dict:
    for rank in tfidf_dict[ngram]:
        np.savetxt(f'csvfiles/tfidf_n{ngram}_r{rank}.csv', tfidf_dict[ngram][rank], delimiter=',')